# Visualization of tif files

In [12]:
import src

In [13]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import matplotlib

from src.utils import acquire_data, load_convert_tiff

In [14]:
allregions = ["accra_20181031", "biscay_20180419", "danang_20181005",
              "kentpointfarm_20180710", "kolkata_20201115", "lagos_20190101",
              "lagos_20200505", "london_20180611", "longxuyen_20181102",
              "mandaluyong_20180314", "neworleans_20200202", "panama_20190425",
              "portalfredSouthAfrica_20180601", "riodejaneiro_20180504",
              "sandiego_20180804", "sanfrancisco_20190219", "shengsi_20190615",
              "suez_20200403", "tangshan_20180130", "toledo_20191221",
              "tungchungChina_20190922", "tunisia_20180715",
              "turkmenistan_20181030", "venice_20180630", "venice_20180928",
              "vungtau_20180423"
    ]

In [15]:
path = "../../../../data/"
ext = ".tif"


for region_name in allregions:
    sentinel_img, coords_dict = acquire_data(path + region_name + ext)
    print(coords_dict['lat'].shape)
    rgb, ndvi, fdi, meta = load_convert_tiff(path + region_name + ext)
    plt.imshow(np.moveaxis(rgb, 0, -1))
    plt.show()



#sentinel_img, coords_dict = acquire_data(path + region_name + ext)
#for i in range(0, 11):
#    plt.imshow(sentinel_img[:, :, i:i+3])
#    plt.show()
#print(coords_dict['lat'].shape)

#rgb, ndvi, fdi, meta = load_convert_tiff(path + region_name + ext)
#plt.imshow(np.moveaxis(rgb, 0, -1))
#plt.show()


(5380, 10283)


KeyboardInterrupt: 